In [0]:
!pip install openai==0.28

In [0]:
dbutils.library.restartPython()

In [0]:
import openai
from pathlib import Path
import os
from ParsingMemoriaCalculo import tabela_texto_memoria_para_df
from pydantic import BaseModel
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2023-12-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea','azure-oai-dk')

In [0]:
#acho que da pra melhorar o prompt\n",
    "\n",
    "ex_dic = {\n",
    "    'ativo': 'UNEG11',\n",
    "    'pior_rating_permitido': 2, \n",
    "    'rating_do_ativo': 6,\n",
    "    'posicao_ativo': 118787475.0,\n",
    "    'descricao_regra': 'Vedação - Ativos de Crédito Privado c/ Rating Diferente de A'\n",
    "}\n",
    "\n",
    "ex_regra = \"\"\"\n",
    "Descrição da Regra:\n",
    "1. Tipo de ativo: Crédito Privado.\n",
    "2. A regra proíbe ativos com rating diferente de A.\n",
    "3. O rating do ativo fornecido é 6, o que viola a regra.\n",
    "4. O ativo em questão é 'UNEG11', e sua posição é 118787475.0.\n",
    "\"\"\"\n",
    "resposta = gerar_restricao(ex_regra,ex_dic)\n",
    "print(resposta)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "324fd083-2f0f-44bb-a82b-7b61817fc6b9",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "#podemos pensar também em como fazer um prompt que permita receber varias instruções e retorne restrições diferentes para cada uma delas \n",
    "def gerar_restricao(mensagem_regra, dados_ativos):\n",
    "    \"\"\"\n",
    "    Função que gera restrições com base na regra e dados dos ativos.\n",
    "    \n",
    "    Parâmetros:\n",
    "    - mensagem_regra (str): Descrição da regra a ser aplicada.\n",
    "    - dados_ativos (dict): Dicionário com os dados do ativo e da regra.\n",
    "    \n",
    "    Retorno:\n",
    "    - (str): Mensagem que descreve a restrição gerada.\n",
    "    \"\"\"\n",
    "    ativo = dados_ativos['ativo']\n",
    "    rating_ativo = dados_ativos['rating_do_ativo']\n",
    "    rating_permitido = dados_ativos['pior_rating_permitido']\n",
    "    descricao_regra = dados_ativos.get('descricao_regra', 'Regra não especificada')\n",
    "\n",
    "    if rating_ativo > rating_permitido:\n",
    "        resposta = f\"O ativo '{ativo}' não cumpre a regra '{descricao_regra}', pois seu rating ({rating_ativo}) excede o limite permitido ({rating_permitido}).\"\n",
    "    else:\n",
    "        resposta = f\"O ativo '{ativo}' está em conformidade com a regra '{descricao_regra}'.\"\n",
    "\n",
    "    return resposta"

In [0]:
#Função para Gerar Restrição
class FormatoResposta(BaseModel):
    codigo: str   
    tipo: str

def gerar_restricao():
    dir_atual = Path(os.getcwd())
    path_final = dir_atual.absolute().parent / Path("ScriptsSQL") / Path("Nexxus_desenquadramento.sql")
    with open (path_final, 'r') as file:
        sql_script = file.read()
    df = spark.sql(sql_script).toPandas()
    parseados = []
    for i,row in df.iterrows():
        parseados.append(tabela_texto_memoria_para_df(row['MemoriaCalculo']))
    exemplo = df.loc[1]
    exemplo_memoria = parseados[1]
    exemplo_memoria = exemplo_memoria[['NOME','POSIÇÃO']].groupby('NOME').sum().reset_index() #Talvez precise do titulo
    fundo = exemplo['IdFundo']
    valor_base = exemplo['SaldoBaseCalculo']
    exposicao = exemplo['ValorExposicao']
    valor_objeto = exemplo['SaldoObjeto']
    l_min = exemplo['LimiteMin']
    l_max = exemplo['LimiteMax']
    descricao = exemplo['DescricaoDetalhada']
    #Pegar ativo pela memória de cálculo -> Usar ativo e fundo para localizar qual posição na régua original vai ser afetada pela restrição
    message_text = [
                {"role":"system",
                "content":
                    """
                    Você deve gerar uma restrição no estilo scipy no formato:
                    def restricao(x):
                        return x
                    Considere 
                    Lembre-se que x é um valor monetário igual valor objeto e não um percentual, sua restrição deve considerar o calculo. Como também considerar os limites mínimos e máximos.
                    Também retorne um texto escrito "eq" ou "ineq" a depender do tipo de restrição que estiver usando.  
                """
                },
                {"role":"user","content":f"Descrição: {descricao}; Limite Mínimo: {l_min}; Limite Máximo: {l_max}; Valor Base: {valor_base}; Valor Objeto: {valor_objeto}, Valor Exposição: {exposicao}"}
               ]
    
    completion = openai.ChatCompletion.create(
        engine="gpt35turbo16k",
        messages = message_text,
        temperature=0,
        max_tokens=1200,
        top_p=0.8,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        )

    return completion.to_dict()['choices'][0]['message']['content']

In [0]:
teste = gerar_restricao()
print(teste)

In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where status = 1 

In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_regras where IdRegra = 636
